<a href="https://colab.research.google.com/github/Anusarakh/thesis/blob/main/Test_Music_genre_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Thesis/music/features_30_sec_2label.csv')
data

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.06450,2002.449060,85882.76132,3805.839606,9.015054e+05,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.07360,2039.036516,213843.75550,3550.522098,2.977893e+06,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.64340,1747.702312,76254.19226,3042.260232,7.840345e+05,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.94240,1596.412872,166441.49480,2184.745799,1.493194e+06,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.93930,1748.172116,88445.20904,3579.757627,1.572978e+06,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.227693,0.088256,0.013564,0.000229,1800.888692,285133.98220,1781.068502,62925.51721,3427.243261,8.645783e+05,...,136.467468,2.417449,125.991013,0.779610,68.256584,-3.415084,162.598373,-0.739651,160.096695,classical
196,0.293052,0.082874,0.056987,0.000786,1933.689602,192223.36290,1740.491088,27385.87471,3573.962147,4.543625e+05,...,49.117809,2.988827,39.963310,3.548611,52.965549,-3.346735,87.178192,-3.686793,108.163559,classical
197,0.222265,0.083254,0.036720,0.000465,1494.419821,92293.65472,1671.195986,31772.32080,2781.813393,3.395294e+05,...,97.051605,0.787177,74.531082,-3.961360,52.424522,-9.148158,118.060761,-6.935011,104.361465,classical
198,0.289919,0.088790,0.029094,0.000332,1952.495511,166304.03580,1865.737617,22529.55381,3779.826564,4.477117e+05,...,59.865459,7.026492,83.698685,8.378774,71.380379,-5.610244,61.358536,1.379220,123.965485,classical


In [4]:
data.shape

(200, 58)

##Encoding the Labels

In [5]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
y 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [6]:
y.shape

(200,)

##Scaling the Feature columns


In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
X

array([[ 0.73728741,  0.14400498,  0.63020804, ..., -0.62101218,
         0.67385914, -0.82467388],
       [ 0.59518898,  1.20096841,  0.09985569, ..., -0.71352288,
         0.52629703, -0.84561476],
       [ 0.94714023, -0.44723371,  1.33170795, ..., -0.8179529 ,
        -0.06441715, -1.12269925],
       ...,
       [-1.24248287, -0.79047867, -0.81646493, ...,  0.85191327,
        -1.07024479,  0.22123602],
       [-0.19462869,  0.14961664, -0.93445298, ..., -0.47381717,
         0.70763007,  0.57829104],
       [ 0.48669101, -1.56651324, -1.22203362, ..., -0.15887977,
         2.54636115,  0.28871141]])

In [8]:
X.shape

(200, 57)

##Dividing data into training and Testing set

In [9]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)
X_traintrain, X_val, y_traintrain, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=6)

In [10]:
len(y_train)

160

In [11]:
len(y_test)

40

In [12]:
len(X_traintrain)

128

#สร้าง Model 

##KNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier

###Define-Train Model

In [18]:
neigh = KNeighborsClassifier(n_neighbors=5, weights='distance') #กำหนด n=5 และอ้างอิงจากข้อมูลที่ใกล้เคียงที่สุด
neigh.fit(X_traintrain,y_traintrain)

KNeighborsClassifier(weights='distance')

##Validate model

In [43]:
from sklearn.metrics import accuracy_score
knn_result = neigh.predict(X_val)
accuracy_score(y_val,knn_result)

0.96875

In [44]:
from sklearn.metrics import classification_report, confusion_matrix

In [45]:
cm_kNN = confusion_matrix(y_val,knn_result)
cm_kNN

array([[20,  1],
       [ 0, 11]])

In [46]:
print(classification_report(knn_result, y_val))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       1.00      0.92      0.96        12

    accuracy                           0.97        32
   macro avg       0.98      0.96      0.97        32
weighted avg       0.97      0.97      0.97        32



##SVM

In [19]:
from sklearn import svm

In [53]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_traintrain,y_traintrain)

#Predict the response for test dataset
svm_result = clf.predict(X_val)

In [54]:
svm_result

array([1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0])

In [40]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.95
Precision: 0.9444444444444444
Recall: 0.9444444444444444


In [55]:
cm_svm = confusion_matrix(y_val,svm_result)
cm_svm

array([[21,  0],
       [ 0, 11]])

In [56]:
print(classification_report(svm_result, y_val))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      1.00      1.00        11

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

